<a href="https://colab.research.google.com/github/Junjaee/Study/blob/main/Python_Data_Analytics_Library/10_%EB%B6%84%EC%84%9D%EC%98%88%EC%A0%9C6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib as mpl
import matplotlib.font_manager as fm
fm.FontProperties(fname = "맑은 고딕")
mpl.rcParams["axes.unicode_minus"] = False # 음수표시

import seaborn as sns

plt.style.use("fivethirtyeight")

import warnings
warnings.filterwarnings("ignore")


import urllib.request as req
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
import requests
# from selenium import webdriver
import urllib.parse

from bs4 import BeautifulSoup

# 셀프 주유소는 과연 저렴한가?

- 실제 셀프 주유소가 저렴한지에 대한 팩트 체크
- 가장 싼 주유소는 어디인가?
- http://www.opinet.co.kr


### 1. 서울시 구별 주유소 가격 정보 가져오기

#### 1) Sample Code

In [ ]:
##### 원격 제어창 실행

!pip install Selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

In [ ]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver', options = chrome_options)
driver.get("https://www.opinet.co.kr/searRgSelect.do")



In [ ]:
# 시 목록 가져오기
driver.get("https://www.opinet.co.kr/searRgSelect.do")

si_list_raw = driver.find_element_by_xpath("""//*[@id="SIDO_NM0"]""")

# si_list = si_list_raw.find_element_by_tag_name("option")#태그 하나만 가저옴
si_list = si_list_raw.find_elements_by_tag_name("option")#태그 전부 가저옴
si_names = [option.get_attribute("value") for option in si_list]
si_names.remove("")
si_names

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [ ]:
#시 선택하기
element = driver.find_element_by_id("SIDO_NM0")
element.send_keys(si_names[0])

In [ ]:
# 구 목록 가져오기

gu_list_raw = driver.find_element_by_xpath("""//*[@id="SIGUNGU_NM0"]""")
#si_list = si_list_raw.find_element_by_tag_name("option")#태그 하나만 가저옴
gu_list = gu_list_raw.find_elements_by_tag_name("option")#태그 전부 가저옴
gu_names=[option.get_attribute("value") for option in gu_list]
gu_names.remove("")
gu_names

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [ ]:
#구 선택하기

element = driver.find_element_by_id("SIGUNGU_NM0")
element.send_keys(gu_names[2])

In [ ]:
# 조회 버튼 누르기 (이 사이트의 경우 자동으로 조회가 눌리기 때문에 안해도 상관없음)

xp = """//*[@id="searRgSelect"]/span"""
driver.find_element_by_xpath(xp).click()

In [ ]:
#엑셀 저장 버튼  누르기

xp = """//*[@id="glopopd_excel"]/span"""
driver.find_element_by_xpath(xp).click()

#### 2) 지역별(구 별)로 주유소 정보를 엑셀파일로 저장

In [ ]:
# 지역별(구별)로 주유소 정보를 엑셀파일로 저장 
driver.get("https://www.opinet.co.kr/searRgSelect.do")

from tqdm import tqdm_notebook
import time

for gu in tqdm_notebook(gu_names):
    elemnet = driver.find_element_by_id("SIGUNGU_NM0")
    elemnet.send_keys(gu)
    
    time.sleep(1)
    
    xp = """//*[@id="searRgSelect"]/span"""
    driver.find_element_by_xpath(xp).click()
    
    time.sleep(1)
    
    xp = """//*[@id="glopopd_excel"]/span"""
    driver.find_element_by_xpath(xp).click()

    time.sleep(1)
    
driver.close()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"/content/drive/My Drive/Colab Notebooks/Study/Python Data Analytics Library/data/분석예제_10"

In [ ]:
# 저장된 엑셀파일 불러오기
from glob import glob

station_files = glob("/content/drive/My Drive/Colab Notebooks/Study/Python Data Analytics Library/data/분석예제_10/*")
station_files

tmp_raw=[]
for fname in station_files:
    tmp = pd.read_excel(fname, header=2)
    tmp_raw.append(tmp)

In [ ]:
tmp_raw

[       지역                     상호                         주소  ...   휘발유    경유  실내등유
 0   서울특별시               오일프러스 셀프    서울 강남구 남부순환로 2651 (도곡동)  ...  1593  1389     -
 1   서울특별시            (주)보성 세곡주유소      서울 강남구  헌릉로 731 (세곡동)  ...  1595  1395     -
 2   서울특별시                  방죽주유소      서울 강남구 밤고개로 215 (율현동)  ...  1607  1409     -
 3   서울특별시      지에스칼텍스(주)직영 산성주유소             서울 강남구 헌릉로 730  ...  1610  1410     -
 4   서울특별시         현대오일뱅크 도곡셀프주유소  서울 강남구  남부순환로 2718 (도곡2동)  ...  1613  1409     -
 5   서울특별시                  대교주유소      서울 강남구 강남대로 640 (신사동)  ...  1615  1415     -
 6   서울특별시        현대오일뱅크(주) 유진주유소       서울 강남구 논현로 152 (도곡동)  ...  1628  1424  1070
 7   서울특별시    지에스칼텍스㈜에너지플러스허브GS타워       서울 강남구 논현로 516 (역삼동)  ...  1628  1428     -
 8   서울특별시                자곡셀프주유소      서울 강남구 밤고개로 120 (자곡동)  ...  1645  1437     -
 9   서울특별시                SK서광주유소             서울 강남구 역삼로 142  ...  1645  1445     -
 10  서울특별시         (주)중앙에너비스 수서지점       서울 강남구 광평로 202 (수서동)  ...  1668  145

In [ ]:
station_raw = pd.concat(tmp_raw)
station_raw
station_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 486 entries, 0 to 14
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      486 non-null    object
 1   상호      486 non-null    object
 2   주소      486 non-null    object
 3   상표      486 non-null    object
 4   전화번호    486 non-null    object
 5   셀프여부    486 non-null    object
 6   고급휘발유   486 non-null    object
 7   휘발유     486 non-null    object
 8   경유      486 non-null    object
 9   실내등유    486 non-null    object
dtypes: object(10)
memory usage: 41.8+ KB


In [ ]:
#데이터프레임 재구성

stations=pd.DataFrame({"Oil_store":station_raw["상호"],
            "주소":station_raw["주소"],
            "가격":station_raw["휘발유"],
            "셀프":station_raw["셀프여부"],
            "상표":station_raw["상표"]})
stations.head()

,Oil_store,주소,가격,셀프,상표
0,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),1593,Y,SK에너지
1,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1595,Y,SK에너지
2,방죽주유소,서울 강남구 밤고개로 215 (율현동),1607,Y,GS칼텍스
3,지에스칼텍스(주)직영 산성주유소,서울 강남구 헌릉로 730,1610,Y,GS칼텍스
4,현대오일뱅크 도곡셀프주유소,서울 강남구 남부순환로 2718 (도곡2동),1613,Y,현대오일뱅크


In [ ]:
#파생변수 추가
stations["구"] = [addr.split()[1] for addr in stations["주소"]]
stations.head()

,Oil_store,주소,가격,셀프,상표,구
0,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),1593,Y,SK에너지,강남구
1,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1595,Y,SK에너지,강남구
2,방죽주유소,서울 강남구 밤고개로 215 (율현동),1607,Y,GS칼텍스,강남구
3,지에스칼텍스(주)직영 산성주유소,서울 강남구 헌릉로 730,1610,Y,GS칼텍스,강남구
4,현대오일뱅크 도곡셀프주유소,서울 강남구 남부순환로 2718 (도곡2동),1613,Y,현대오일뱅크,강남구


In [ ]:
stations["구"].unique()

array(['강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구',
       '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구', '송파구',
       '양천구', '영등포구', '용산구', '은평구', '종로구', '중구', '중랑구'], dtype=object)

In [ ]:
#결측치 처리

stations[stations["가격"] == "-"]
stations = stations[stations["가격"]!= "-"]

In [ ]:
stations[stations["가격"] == "-"]

,Oil_store,주소,가격,셀프,상표,구


In [ ]:
#인덱스 재정렬
stations.reset_index(inplace=True)

In [ ]:
stations.head()

,index,Oil_store,주소,가격,셀프,상표,구
0,0,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),1593,Y,SK에너지,강남구
1,1,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1595,Y,SK에너지,강남구
2,2,방죽주유소,서울 강남구 밤고개로 215 (율현동),1607,Y,GS칼텍스,강남구
3,3,지에스칼텍스(주)직영 산성주유소,서울 강남구 헌릉로 730,1610,Y,GS칼텍스,강남구
4,4,현대오일뱅크 도곡셀프주유소,서울 강남구 남부순환로 2718 (도곡2동),1613,Y,현대오일뱅크,강남구


In [ ]:
#index 컬럼삭제

del stations["index"]

In [ ]:
stations.head()

,Oil_store,주소,가격,셀프,상표,구
0,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),1593,Y,SK에너지,강남구
1,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1595,Y,SK에너지,강남구
2,방죽주유소,서울 강남구 밤고개로 215 (율현동),1607,Y,GS칼텍스,강남구
3,지에스칼텍스(주)직영 산성주유소,서울 강남구 헌릉로 730,1610,Y,GS칼텍스,강남구
4,현대오일뱅크 도곡셀프주유소,서울 강남구 남부순환로 2718 (도곡2동),1613,Y,현대오일뱅크,강남구


In [ ]:
#휘발유 컬럼 형변환 문자 -> 숫자
stations["가격"] = [float(val) for val in stations["가격"]]
stations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 474 entries, 0 to 473
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Oil_store  474 non-null    object 
 1   주소         474 non-null    object 
 2   가격         474 non-null    float64
 3   셀프         474 non-null    object 
 4   상표         474 non-null    object 
 5   구          474 non-null    object 
dtypes: float64(1), object(5)
memory usage: 25.9+ KB


Object `read.csv` not found.
